**Learning Objective:**
Learn how to apply linear mapping concepts on removing projective distortion

**Problem:**
When an image is taken by the camera, there is an inherent projective distortion. This makes rectangular objects appear distorted. However, this projective distortion can be reversed if we know the 3x3 matrix that maps from projective to affine (frontal no distortion). 

Using our knowledge on linear mapping and least squares estimation, develop a program that will remove the projective distortion on a given image. Note that at least 4 pts on the target undistorted image must be known.

In [1]:
import numpy as np
import cv2

In [2]:
# function to get the four points of the distorted image clicked (by mouse) by the user
def selectSource(event, x, y, flags, param):
    global src, orig 
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(src) < 4:
            cv2.circle(orig , (x, y), 5, (255, 0, 0), -1)
            if len(src) > 0:
                cv2.line(orig, (x, y), (src[-1][0], src[-1][1]), (255, 0, 0), 3)
            src.append([x, y])
        
        if len(src) == 4:
            cv2.line(orig, (src[-1][0], src[-1][1]), (src[0][0], src[0][1]), (255, 0, 0), 3)

In [3]:
# function to compute for the homography that maps from projective to affine
def getHomography(src, dst):
    A = []
    for i in range(0, len(src)):
        x, y = src[i][0], src[i][1]
        u, v = dst[i][0], dst[i][1]
        A.append([-x, -y, -1, 0, 0, 0, u * x, u * y, u])
        A.append([0, 0, 0, -x, -y, -1, v * x, v * y, v])
    lastRow = [0, 0, 0, 0, 0, 0, 0, 0, 1]
    A.append(lastRow)
    A = np.asarray(A)
    hvec = np.vstack(lastRow)
    H = np.matmul(np.linalg.inv(A), hvec)
    H = H.reshape(3,3)
    return H

In [12]:
# function for image mapping (not used)
def projectiveToAffine(height, width, image, homography):
    
    #empty image
    transformed = np.zeros_like(image)

    for i in range(height):
        for j in range(width):
            k = np.array([i, j, 1])
            dst = np.dot(homography, k)
            dst = dst / dst[-1]
            i_dst, j_dst = round(dst[0]), round(dst[1])
            if 0 <= i_dst < height and 0 <= j_dst < width:
                transformed[i_dst, j_dst] = image[i, j]
    return transformed

**Main.** Change the *'test1.png'* to the filename of the distorted image *(should be on the same folder as this program)* of your choice. The program reads and displays this input image from the user. From there, select the section of the image you want to map to affine by clicking four points **_(clockwise, starting from top left)_** in the image. By clicking the **'r' key**, you can restart the selection of points; by clicking **'esc'** key, you can exit from the displayed window. After you select a section, click the **'t' key** so that the program displays the undistorted version of the image.

In [55]:
src = []

orig = cv2.imread(r'test1.png')
#orig = cv2.imread(r'test2.png')

try:
    if orig.shape[0] > 700 and orig.shape[1] > 700:
        orig = cv2.resize(orig, (0,0), fx = 0.35, fy = 0.35)
except Exception as e:
    print(str(e))
    
clone = orig.copy()
h, w = orig.shape[:2]
dst = [[0,0], [w-1, 0], [w-1, h-1], [0, h-1]]

cv2.namedWindow('Original Image')
cv2.setMouseCallback('Original Image', selectSource)

while(1):
    cv2.imshow('Original Image', orig)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('t'):
        H = getHomography(src, dst)
        transformed = cv2.warpPerspective(clone, H, (w, h), flags=cv2.INTER_AREA)
        cv2.imshow('Corrected Image', transformed)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        break
    elif key == ord('r'):    #restart source points
        orig = clone.copy()
        src = []
    elif key == 27:    #press Esc to exit
        break
cv2.destroyAllWindows()